In [108]:
import pandas as pd
import numpy as np

In [109]:
df = pd.read_csv('titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [110]:
df=df.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)


In [111]:
from sklearn.model_selection import train_test_split
x = df.drop(['Survived'],axis=1)
y = df[['Survived']]

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)
x_train.shape , x_test.shape

((623, 7), (268, 7))

In [112]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 623 entries, 505 to 84
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    623 non-null    int64  
 1   Sex       623 non-null    object 
 2   Age       498 non-null    float64
 3   SibSp     623 non-null    int64  
 4   Parch     623 non-null    int64  
 5   Fare      623 non-null    float64
 6   Embarked  621 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 38.9+ KB


In [113]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

### handling missing values

In [114]:
from sklearn.impute import SimpleImputer
# for age and embarked
si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')

x_train_age = si_age.fit_transform(x_train[['Age']])
x_test_age = si_age.transform(x_test[['Age']])

x_train_embarked = si_embarked.fit_transform(x_train[['Embarked']])
x_test_embarked = si_embarked.transform(x_test[['Embarked']])
x_test_embarked.shape

(268, 1)

### one hot encoding

In [115]:
from sklearn.preprocessing import OneHotEncoder
# for sex and embarked 

ohe_sex = OneHotEncoder(dtype='int',sparse_output=False,drop='first')
ohe_embarked = OneHotEncoder(dtype='int',sparse_output=False,drop='first')

x_train_sex = ohe_sex.fit_transform(x_train[['Sex']])
x_test_sex = ohe_sex.transform(x_test[['Sex']])

x_train_embarked = ohe_embarked.fit_transform(x_train_embarked)
x_test_embarked = ohe_embarked.transform(x_test_embarked)

In [116]:
ohe_embarked.categories_

[array(['C', 'Q', 'S'], dtype=object)]

In [117]:
x_train_remain = x_train.drop(['Age','Sex','Embarked'],axis=1).values
x_test_remain = x_test.drop(['Age','Sex','Embarked'],axis=1).values
# ['Survived', 'Pclass', 'SibSp', 'Parch', 'Fare']
x_test_remain.shape,x_test_age.shape,x_test_age.shape,x_test_embarked.shape

((268, 4), (268, 1), (268, 1), (268, 2))

In [118]:
x_train_transformed = np.concatenate([x_train_remain,x_train_age,x_train_sex,x_train_embarked],axis=1)
x_test_transformed = np.concatenate([x_test_remain,x_test_age,x_test_sex,x_test_embarked],axis=1)

In [119]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(x_train_transformed,y_train)

DecisionTreeClassifier()

In [120]:
y_pred = clf.predict(x_test_transformed)

In [121]:
from sklearn.metrics import accuracy_score

print("The Model Accuracy is : ",accuracy_score(y_test,y_pred))

The Model Accuracy is :  0.7910447761194029


### Converting into binary files

In [122]:
import pickle

pickle.dump(ohe_sex,open('models/ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked,open('models/ohe_embarked.pkl','wb'))
pickle.dump(clf,open('models/clf.pkl','wb'))